In [166]:
# 전체 라이브러리 모아두기
import pandas as pd
import os
import sentencepiece as spm
import json
import re
import glob

In [ ]:
def clean_text(text):
    return re.sub(r'#@\S+', '', text).strip()

In [94]:
def train_sentencepiece(input_file, model_dir="../model/llm_model", model_name="chatbot_spm", vocab_size=16000):

    model_prefix = os.path.join(model_dir, model_name)

    spm.SentencePieceTrainer.Train(
        f"--input={input_file} --model_prefix={model_prefix} --vocab_size={vocab_size} "
        "--model_type=bpe --character_coverage=1.0 --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3"
    )

    print(f"✅ 모델 저장 완료: {model_prefix}.model")
    print(f"✅ 단어 사전 저장 완료: {model_prefix}.vocab")

In [ ]:
# def save_for_spm_training(pairs, out_path):
#     with open(out_path, "w", encoding="utf-8") as f:
#         for pair in pairs:
#             f.write(pair["input"].strip() + "\n")
#             f.write(pair["target"].strip() + "\n")

# # 예시 사용
# with open("../data/converted_dataset/train_chatbot_data.json", "r", encoding="utf-8") as f:
#     data = json.load(f)

# save_for_spm_training(data, "../data/text_dataset/text_for_txt/train.txt")

KeyboardInterrupt: 

In [ ]:
# 실행 예시
# train_sentencepiece("../data/text_dataset/text_for_txt/train.txt", model_dir="../model/llm_model", model_name="chatbot_spm")

✅ 모델 저장 완료: ../model/llm_model/chatbot_spm.model
✅ 단어 사전 저장 완료: ../model/llm_model/chatbot_spm.vocab


In [167]:
class SentencePieceTokenizer:
    def __init__(self, model_path):
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(model_path)
        self.vocab_size = self.sp.get_piece_size()
        
    def encode(self, text):
        return self.sp.encode(text, out_type=int)
    
    def decode(self, ids):
        return self.sp.decode(ids)
    
    def pad_id(self):
        return self.sp.pad_id()
    
    def bos_id(self):
        return self.sp.bos_id()
    
    def eos_id(self):
        return self.sp.eos_id()

In [169]:
# 테스트
tokenizer = SentencePieceTokenizer("../model/llm_model/chatbot_spm.model")

ids = tokenizer.encode("알아서 하던가")
print("tokenized:", ids)

text = tokenizer.decode(ids)
print("decode:", text)

print("PAD ID:", tokenizer.pad_id())

print(tokenizer.encode("안녕"))
print(tokenizer.decode(tokenizer.encode('안녕')))

tokenized: [1583, 24, 1560]
decode: 알아서 하던가
PAD ID: 0
[2688]
안녕


In [170]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [173]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=2, dropout=0.1):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, 
                            batch_first=True, dropout=dropout, bidirectional=False)
        
    def forward(self, x):
        embeded = self.embedding(x)
        outputs, (hidden, cell) = self.lstm(embeded)
        return outputs, (hidden, cell)

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        if hidden.dim() == 2:
            hidden = hidden.unsqueeze(1)

        # batch_size = encoder_outputs.size(0)
        seq_len = encoder_outputs.size(1)
        hidden = hidden.repeat(1, seq_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        attention = self.v(energy).squeeze(2)
        attn_weight = F.softmax(attention, dim=1)
        context = torch.bmm(attn_weight.unsqueeze(1), encoder_outputs)
        return context, attn_weight

class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers=2, dropout=0.1):
        super(Decoder, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size + hidden_size, hidden_size,
                            num_layers, batch_first=True, dropout=dropout)
        self.fc_out = nn.Linear(hidden_size * 2, vocab_size)
        self.attention = Attention(hidden_size)

    def forward(self, input_token, hidden, cell, encoder_outputs):
        if input_token.dim() == 1:
            input_token = input_token.unsqueeze(1)

        embedded = self.embedding(input_token)
        context, attn_weights = self.attention(hidden[-1], encoder_outputs)
        lstm_input = torch.cat((embedded, context), dim=2)

        outputs, (hidden, cell) = self.lstm(lstm_input, (hidden, cell))
        concat = torch.cat((outputs, context), dim=2)
        logits = self.fc_out(concat).squeeze(1)

        return logits, hidden, cell, attn_weights

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size, trg_len = trg.shape
        vocab_size = self.decoder.embedding.num_embeddings

        outputs = torch.zeros(batch_size, trg_len, vocab_size).to(self.device)

        encoder_outputs, (hidden, cell) = self.encoder(src)
        input_token = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell, _ = self.decoder(input_token, hidden, cell, encoder_outputs)
            outputs[:, t] = output
            top1 = output.argmax(1)
            input_token = trg[:, t] if torch.rand(1).item() < teacher_forcing_ratio else top1

        return outputs

In [ ]:
import torch
from torch.utils.data import Dataset
import json

class ChatDataset(Dataset):
    def __init__(self, json_path, tokenizer, max_len=64):
        with open(json_path, "r", encoding="utf-8") as f:
            self.data = json.load(f)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        input_text = clean_text(self.data[idx]["input"])
        target_text = clean_text(self.data[idx]["target"])

        input_ids = [self.tokenizer.bos_id()] + self.tokenizer.encode(input_text) + [self.tokenizer.eos_id()]
        target_ids = [self.tokenizer.bos_id()] + self.tokenizer.encode(target_text) + [self.tokenizer.eos_id()]

        # 패딩 처리
        input_ids = input_ids[:self.max_len] + [self.tokenizer.pad_id()] * max(0, self.max_len - len(input_ids))
        target_ids = target_ids[:self.max_len] + [self.tokenizer.pad_id()] * max(0, self.max_len - len(target_ids))

        return torch.tensor(input_ids), torch.tensor(target_ids)

In [175]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [113]:
def train_model(model, dataloader, tokenizer, num_epochs=5, lr=1e-3, device=None, checkpoint_path=None):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    # gpu 기반
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    pad_id = tokenizer.pad_id()
    criterion = nn.CrossEntropyLoss(ignore_index=pad_id)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    start_epoch = 0
    if checkpoint_path is not None and os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        start_epoch = checkpoint["epoch"] + 1
        print(f"🔄 체크포인트에서 재시작: epoch {start_epoch}")
    else:
        print("🆕 새로 학습을 시작합니다.")

    model.train()

    for epoch in range(start_epoch, num_epochs):
        epoch_loss = 0
        progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for src, trg in progress_bar:
            src, trg = src.to(device), trg.to(device)
            optimizer.zero_grad()

            output = model(src, trg)
            output_dim = output.shape[-1]

            output = output[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)

            loss = criterion(output, trg)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())

        avg_loss = epoch_loss / len(dataloader)
        print(f"\n[Epoch {epoch+1}] 평균 Loss: {avg_loss:.4f}")

        # checkpoint 저장
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, f"checkpoint_epoch{epoch}.pt")

    return model

### 모델 저장 및 평가

In [176]:
def save_model(model, path='../model/llm_model/chatbot_model.pt'):
    torch.save(model.state_dict(), path)
    print(f"✅ 모델 저장 완료: {path}")

def load_model(model, path='../model/llm_model/chatbot_model.pt'):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model.load_state_dict(torch.load(path, map_location=device))
    model.to(device)
    model.eval()
    return model

In [177]:
from torch.nn.functional import softmax

@torch.no_grad()
def evalute_model(model, dataloader, tokenizer,
                    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")):
    # device='cuda' if torch.cuda.is_available() else 'cpu'
    model.eval()
    model.to(device)

    for i, (src, trg) in enumerate(dataloader):
        if i >= 5: break

        src, trg = src.to(device), trg.to(device)

        # infernce 모드
        output = model(src, trg, teacher_forcing_ratio=0.0)
        pred = output.argmax(dim=-1)

        print("🟢 Input :", tokenizer.decode(src[0].tolist()))
        print("✅ Target :", tokenizer.decode(trg[0].tolist()))
        print("🤖 Output :", tokenizer.decode(pred[0].tolist()))
        print("-"*60)

In [178]:
def generate_reply(model, tokenizer, input_text, max_len=64, 
                device=torch.device("mps" if torch.backends.mps.is_available() else "cpu")):
    model.eval()
    model.to(device)

    # 입력 인코딩
    input_ids = tokenizer.encode(input_text)
    input_ids = input_ids + [tokenizer.pad_id()] * (max_len - len(input_ids))
    src = torch.tensor(input_ids).unsqueeze(0).to(device)  # [1, seq_len]

    input_token = torch.tensor([tokenizer.bos_id()], device=device)  # 시작 토큰 호출 O

    generated_ids = []

    with torch.no_grad():
        # 인코더 출력
        encoder_outputs, (hidden, cell) = model.encoder(src)

        for t in range(max_len):
            output, hidden, cell, _ = model.decoder(input_token.view(1, 1), hidden, cell, encoder_outputs)

            if t < 5:
                output[0][tokenizer.eos_id()] = -float('inf')  # 호출 O

            next_token = output.argmax(1)

            if next_token.item() == tokenizer.eos_id():  # 호출 O
                break

            generated_ids.append(next_token.item())
            input_token = next_token

    decoded = tokenizer.decode(generated_ids)

    print("input_ids:", input_ids)
    print("generated_ids:", generated_ids)
    print("decoded:", decoded)
    
    return decoded

### 테스트

In [179]:
def interactive_chat(model, tokenizer, device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")):
    # device='cuda' if torch.cuda.is_available() else 'cpu'
    print("🤖 챗봇 테스트 시작. 종료하려면 exit 입력.")

    while(True):
        query = input("👤 사용자: ")
        print("👤 사용자: ", query)
        if query.strip().lower() in ['exit', 'quit', '종료']:
            print("🔴 테스트 종료.")
            break
        response = generate_reply(model, tokenizer, query, device=device)
        print("🤖 챗봇: ", response)

### 전체 파이프라인

In [118]:
def full_pipeline(train_loader, valid_loader, tokenizer, model, num_epochs=5):
    trained_model = train_model(model, train_loader, tokenizer, num_epochs=num_epochs)

    save_model(trained_model, "model/llm_model/chatbot_model.pt")

    evalute_model(trained_model, valid_loader, tokenizer)

    interactive_chat(trained_model, tokenizer)

In [180]:
def test_pipeline(valid_loader, tokenizer, model):
    
    test_model = load_model(model)

    evalute_model(test_model, valid_loader, tokenizer)

    interactive_chat(test_model, tokenizer)

In [ ]:
from torch.utils.data import DataLoader

train_dataset = ChatDataset("../data/converted_dataset/train_chatbot_data.json", tokenizer)
valid_dataset = ChatDataset("../data/converted_dataset/valid_chatbot_data.json", tokenizer)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, drop_last=True)

### 학습 시작 및 모델 저장

In [ ]:
vocab_size = tokenizer.vocab_size
embed_size = 256
hidden_size = 512

encoder = Encoder(vocab_size, embed_size, hidden_size)
decoder = Decoder(vocab_size, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder, device = torch.device("mps" if torch.backends.mps.is_available() else "cpu"))
# gpu 기반
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 전체 파이프라인
# full_pipeline(train_loader, valid_loader, tokenizer, model, num_epochs=5)

# 테스트만
test_pipeline(valid_loader, tokenizer, model)

🟢 Input : 사쓰던가
✅ Target : 가서생각해
🤖 Output : #@
------------------------------------------------------------
🟢 Input : 모자에털있는것으로무사히사땈ᄏᄏᄏ
✅ Target : ᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏᄏ
🤖 Output : #@
------------------------------------------------------------
🟢 Input : 난 126...
✅ Target : 1,2중에ᅦ하나살까
🤖 Output : #@
------------------------------------------------------------
🟢 Input : 라코스테 가방은 또 너무 여름용이야
✅ Target : 왜
🤖 Output : #@
------------------------------------------------------------
🟢 Input : 똑같은거야 ᄏᄏᄏ
✅ Target : 다른거야
🤖 Output : #@
------------------------------------------------------------
🤖 챗봇 테스트 시작. 종료하려면 exit 입력.
👤 사용자:  안녕
input_ids: [2688, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
generated_ids: [10, 48, 6475, 65, 6475]
decoded: #@시스템#사진#
🤖 챗봇:  #@시스템#사진#
👤 사용자:  exit
🔴 테스트 종료.
